# Homework 2: Discovery of Frequent Itemsets and Association Rules

In [1]:
from itertools import combinations
from operator import itemgetter
import pandas as pd
from time import time

In [2]:
#Retail dataset downloaded from http://fimi.uantwerpen.be/data/
dataset = pd.read_csv('small_data')
dataset.head()

,product
0,"30,31,32"
1,"33,34,35"
2,"36,37,38,39,40,41,42,43,44,45,46"
3,"38,39,47,48"
4,"38,39,48,49,50,51,52,53,54,55,56,57,58"


# Frequent Itemsets

In [3]:
def apriori(data, support):

    #Getting all the single items >= the support per column
    single_items = (data['product'].str.split(",", expand=True))\
        .apply(pd.value_counts).sum(axis=1).where(lambda value: value > support).dropna()

    #Appending the list from single_items
    apriori_data = pd.DataFrame(
        {'product': single_items.index.astype(int), 'support': single_items.values, 'set_size': 1})

    #Column for generating subsets
    data['set_size'] = data['product'].str.count(",") + 1

    #Just making this integers
    data['product'] = data['product'].apply(lambda row: set(map(int, row.split(","))))

    #The parent set
    single_items_set = set(single_items.index.astype(int))

    #Starting loop to discount records with one prodcut/item
    for length in range(2, len(single_items_set) + 1):
        data = data[data['set_size'] >= length]
        d = data['product'] \
            .apply(lambda st: pd.Series(s if set(s).issubset(st) else None for s in combinations(single_items_set, length))) \
            .apply(lambda col: [col.dropna().unique()[0], col.count()] if col.count() >= support else None).dropna()
        if d.empty: #If the series is empty don't append and break loop
            break
        apriori_data = apriori_data.append(pd.DataFrame(
            {'product': list(map(itemgetter(0), d.values)), 'support': list(map(itemgetter(1), d.values)),
             'set_size': length}), ignore_index=True)

    return apriori_data

In [ ]:
support=5 #The smaller, the more frequent itemsets we will have

frequent_itemsets=apriori(dataset, support)
frequent_itemsets
